<a href="https://colab.research.google.com/github/youssefkamel02/AI-milestone/blob/main/milestone2_infosec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
class PlayfairCipher:
    """
    Implements the Playfair Cipher for password encryption.
    Note: 'J' is treated as 'I' to fit the 25-letter key square.
    """

    # Static property for the alphabet used (excluding J)
    ALPHABET = "ABCDEFGHIKLMNOPQRSTUVWXYZ"

    def __init__(self, key):
        """Initializes the cipher with a key and generates the 5x5 key table."""
        self.key_table = self._generate_key_table(key.upper())

    def _generate_key_table(self, key):
        """Generates the 5x5 key square from the key."""
        unique_key = ""

        # 1. Process key: Remove non-alpha, replace J with I, keep unique characters
        processed_key = key.replace("J", "I")
        for char in processed_key:
            if 'A' <= char <= 'Z' and char not in unique_key:
                unique_key += char

        # 2. Fill with remaining alphabet (excluding J)
        table_key_str = unique_key
        for char in self.ALPHABET:
            if char not in table_key_str:
                table_key_str += char

        # 3. Map the 25-character string into the 5x5 list of lists
        key_table = []
        for i in range(5):
            row = list(table_key_str[i*5 : (i+1)*5])
            key_table.append(row)

        return key_table

    def _prepare_text(self, plaintext):
        """
        Prepares the plaintext:
        1. Clean and convert to uppercase.
        2. Replace 'J' with 'I'.
        3. Insert filler 'X' between double letters.
        4. Pad with 'X' if the final length is odd.
        """
        # 1 & 2. Clean and handle 'J'
        prepared = "".join(c for c in plaintext.upper() if 'A' <= c <= 'Z').replace("J", "I")

        processed_text = ""
        i = 0
        while i < len(prepared):
            processed_text += prepared[i]

            if i + 1 < len(prepared):
                # 3. Insert filler 'X' if two consecutive letters are the same
                if prepared[i] == prepared[i+1]:
                    processed_text += 'X'
                else:
                    processed_text += prepared[i+1]
                    i += 1 # Move to the next pair element

            i += 1 # Move to the start of the next pair

        # 4. Pad with 'X' if final length is odd
        if len(processed_text) % 2 != 0:
            processed_text += 'X'

        return processed_text

    def _get_char_coords(self, char):
        """Finds the (row, col) coordinates of a character in the key table."""
        for r in range(5):
            if char in self.key_table[r]:
                c = self.key_table[r].index(char)
                return r, c
        return -1, -1 # Should not happen if text is properly prepared

    def encrypt(self, plaintext):
        """Encrypts the password using the Playfair rules."""
        prepared_text = self._prepare_text(plaintext)
        ciphertext = ""

        # Process the text in digraphs (pairs of letters)
        for i in range(0, len(prepared_text), 2):
            char1 = prepared_text[i]
            char2 = prepared_text[i+1]
            r1, c1 = self._get_char_coords(char1)
            r2, c2 = self._get_char_coords(char2)

            new_char1 = ''
            new_char2 = ''

            # Rule 1: Same Row
            if r1 == r2:
                # Shift right (modulo 5 for wrap-around)
                new_char1 = self.key_table[r1][(c1 + 1) % 5]
                new_char2 = self.key_table[r2][(c2 + 1) % 5]

            # Rule 2: Same Column
            elif c1 == c2:
                # Shift down (modulo 5 for wrap-around)
                new_char1 = self.key_table[(r1 + 1) % 5][c1]
                new_char2 = self.key_table[(r2 + 1) % 5][c2]

            # Rule 3: Rectangle
            else:
                # Use the letters on the same row, but in the opposite pair's column
                new_char1 = self.key_table[r1][c2]
                new_char2 = self.key_table[r2][c1]

            ciphertext += new_char1 + new_char2

        return ciphertext

# --- Demonstration ---
if __name__ == "__main__":
    # Example usage for the password encryption milestone
    key = "MONARCHY"
    password = "My Password is SStrong"

    playfair = PlayfairCipher(key)

    print("--- Playfair Cipher Demonstration ---")

    # Optional: Print the key table
    print(f"Key: {key}")
    print("Key Table (5x5):")
    for row in playfair.key_table:
        print(row)
    print("-" * 35)

    prepared_password = playfair._prepare_text(password)
    ciphertext = playfair.encrypt(password)

    print(f"Original Password: {password}")
    print(f"Prepared Text (Digraphs): {prepared_password}")
    print(f"Encrypted Password: {ciphertext}")

--- Playfair Cipher Demonstration ---
Key: MONARCHY
Key Table (5x5):
['M', 'O', 'N', 'A', 'R']
['C', 'H', 'Y', 'B', 'D']
['E', 'F', 'G', 'I', 'K']
['L', 'P', 'Q', 'S', 'T']
['U', 'V', 'W', 'X', 'Z']
-----------------------------------
Original Password: My Password is SStrong
Prepared Text (Digraphs): MYPASXSWORDISXSXSTRONG
Encrypted Password: NCSOXAQXNMBKXAXATLMNYQ


In [2]:
class VigenereCipher:
    """
    Implements the Vigenère Cipher for password encryption.
    It supports encryption and decryption of alphabetic characters.
    """

    def __init__(self, key):
        """Initializes the cipher with the keyword."""
        self.key = self._clean_key(key)

    def _clean_text(self, text):
        """Cleans text to include only uppercase alphabetic characters."""
        return "".join(c for c in text.upper() if 'A' <= c <= 'Z')

    def _clean_key(self, key):
        """Cleans and validates the key."""
        cleaned_key = "".join(c for c in key.upper() if 'A' <= c <= 'Z')
        if not cleaned_key:
            raise ValueError("Vigenere key must contain at least one alphabetic character.")
        return cleaned_key

    def _generate_repeated_key(self, plaintext):
        """Repeats the cipher keyword cyclically to match the plaintext length."""
        key_length = len(self.key)

        # Calculate how many times the key needs to be repeated and the remainder
        full_repeats = len(plaintext) // key_length
        remainder = len(plaintext) % key_length

        # Build the repeated key string
        repeated_key = (self.key * full_repeats) + self.key[:remainder]
        return repeated_key

    def encrypt(self, plaintext):
        """
        Encrypts the plaintext (password) using the Vigenère Cipher.
        Formula: C_i = (P_i + K_i) mod 26
        """
        cleaned_plaintext = self._clean_text(plaintext)
        repeated_key = self._generate_repeated_key(cleaned_plaintext)
        ciphertext = []

        for p_char, k_char in zip(cleaned_plaintext, repeated_key):
            # Convert letters to 0-25 (A=0, B=1, ...)
            p_val = ord(p_char) - ord('A')
            k_val = ord(k_char) - ord('A')

            # Apply encryption formula
            c_val = (p_val + k_val) % 26

            # Convert back to letter
            c_char = chr(c_val + ord('A'))
            ciphertext.append(c_char)

        return "".join(ciphertext)

    def decrypt(self, ciphertext):
        """
        Decrypts the ciphertext using the Vigenère Cipher.
        Formula: P_i = (C_i - K_i + 26) mod 26
        Note: The +26 handles potential negative results from C_i - K_i.
        """
        cleaned_ciphertext = self._clean_text(ciphertext)
        repeated_key = self._generate_repeated_key(cleaned_ciphertext)
        plaintext = []

        for c_char, k_char in zip(cleaned_ciphertext, repeated_key):
            # Convert letters to 0-25
            c_val = ord(c_char) - ord('A')
            k_val = ord(k_char) - ord('A')

            # Apply decryption formula
            p_val = (c_val - k_val + 26) % 26

            # Convert back to letter
            p_char = chr(p_val + ord('A'))
            plaintext.append(p_char)

        return "".join(plaintext)

# --- Demonstration ---
if __name__ == "__main__":
    # Example usage for the password encryption milestone
    key = "SECURITY"
    password = "This is a secret password"

    vigenere = VigenereCipher(key)

    print("--- Vigenère Cipher Demonstration ---")
    print(f"Key: {key}")
    print(f"Original Password: {password}")

    ciphertext = vigenere.encrypt(password)
    print(f"Encrypted Password: {ciphertext}")

    decrypted_text = vigenere.decrypt(ciphertext)
    print(f"Decrypted Password: {decrypted_text}")

--- Vigenère Cipher Demonstration ---
Key: SECURITY
Original Password: This is a secret password
Encrypted Password: LLKMZATQWGTYKXTQKAQLU
Decrypted Password: THISISASECRETPASSWORD


In [6]:
# In a real project, you would install this library: pip install argon2-cffi

from argon2 import PasswordHasher
from argon2.exceptions import VerifyMismatchError

class Argon2PasswordManager:
    """
    Implements Argon2 for secure, one-way password hashing and verification.
    This fulfills the requirement for a modern, suitable password security methodology.
    """

    def __init__(self):
        # Initialize the PasswordHasher with default or custom parameters
        # Argon2 parameters (time_cost, memory_cost, parallelism) can be tuned
        # based on security requirements and server resources.
        self.ph = PasswordHasher()

    def hash_password(self, password: str) -> str:
        """
        Hashes the password using Argon2. The resulting hash includes the
        algorithm, cost parameters, and the salt, all securely encoded.
        """
        # Argon2 handles salt generation internally, which is ideal.
        return self.ph.hash(password)

    def verify_password(self, password: str, hashed_password: str) -> bool:
        """
        Verifies a plaintext password against a stored Argon2 hash.
        Returns True if they match, False otherwise.
        """
        try:
            self.ph.verify(hashed_password, password)

            # Rehash check: If Argon2 parameters were updated, this will check
            # if the hash needs to be recomputed with better settings.
            if self.ph.check_needs_rehash(hashed_password):
                print("[INFO] Password hash needs update (rehash).")
                # In a real app, you would re-save the hash using the latest settings

            return True
        except VerifyMismatchError:
            # This exception is raised if the password does not match the hash
            return False
        except Exception as e:
            # Handle other errors (e.g., hash corruption)
            print(f"[ERROR] Verification failed: {e}")
            return False

# --- Demonstration ---
if __name__ == "__main__":
    manager = Argon2PasswordManager()

   # password = "MySecr3tPassword!"
    password = "youssef"
    # 1. Store the password: Only the hash is stored
    password_hash = manager.hash_password(password)

    print("--- Argon2 Hashing Demonstration ---")
    print(f"Original Password: {password}")
    print(f"Stored Hash (includes salt and params): {password_hash}")

    # 2. Verification attempt 1 (Correct Password)
    is_correct = manager.verify_password(password, password_hash)
    print(f"\nVerification with CORRECT password: {'SUCCESS' if is_correct else 'FAIL'}")

    # 3. Verification attempt 2 (Incorrect Password)
    incorrect_password = "MySecr3tPassword?"
    is_correct = manager.verify_password(incorrect_password, password_hash)
    print(f"Verification with INCORRECT password: {'SUCCESS' if is_correct else 'FAIL'}")

--- Argon2 Hashing Demonstration ---
Original Password: youssef
Stored Hash (includes salt and params): $argon2id$v=19$m=65536,t=3,p=4$L1HPHXmoOZEV7GIx00At0A$i6XKfxWKVsRojmVvFWyjE8+4fo5/f90M4yXhx73a9Q4

Verification with CORRECT password: SUCCESS
Verification with INCORRECT password: FAIL


In [4]:
pip install argon2-cffi

In [8]:
# ====================================================================
# BINF711 - Milestone 2: Cryptography and Encryption Techniques
# Complete Python Implementation (Playfair, Vigenère, and Argon2)
# ====================================================================

# NOTE: Requires external library: pip install argon2-cffi

try:
    from argon2 import PasswordHasher
    from argon2.exceptions import VerifyMismatchError
    # Flag to indicate if the real Argon2 library is available
    ARGON2_AVAILABLE = True
except ImportError:
    # --- Mock classes for environments without argon2-cffi ---
    class PasswordHasher:
        # Mock hash returns a string that lets us know it's a mock
        def hash(self, password): return f"$MOCK2id$v=19$m=65536,t=3,p=4${password[:8]}..."
        # Mock verify always raises an error if the password is wrong
        def verify(self, hashed_password, password):
            # Simple check to simulate correct/wrong verification
            if "wrong" in password:
                 raise VerifyMismatchError
            return True
        def check_needs_rehash(self, hashed_password): return False
    class VerifyMismatchError(Exception): pass
    ARGON2_AVAILABLE = False


# ------------------------------------
# 1. Playfair Cipher Implementation
# ------------------------------------

class PlayfairCipher:
    """Implements the Playfair Cipher. Note: 'J' is treated as 'I'."""
    ALPHABET = "ABCDEFGHIKLMNOPQRSTUVWXYZ"

    def __init__(self, key):
        self.key_table = self._generate_key_table(key.upper())

    def _generate_key_table(self, key):
        unique_key = ""
        processed_key = key.replace("J", "I")
        for char in processed_key:
            if 'A' <= char <= 'Z' and char not in unique_key:
                unique_key += char

        table_key_str = unique_key
        for char in self.ALPHABET:
            if char not in table_key_str:
                table_key_str += char

        key_table = []
        for i in range(5):
            row = list(table_key_str[i*5 : (i+1)*5])
            key_table.append(row)

        return key_table

    def _prepare_text(self, plaintext):
        prepared = "".join(c for c in plaintext.upper() if 'A' <= c <= 'Z').replace("J", "I")
        processed_text = ""
        i = 0
        while i < len(prepared):
            processed_text += prepared[i]

            if i + 1 < len(prepared):
                if prepared[i] == prepared[i+1]:
                    processed_text += 'X'
                else:
                    processed_text += prepared[i+1]
                    i += 1

            i += 1

        if len(processed_text) % 2 != 0:
            processed_text += 'X'

        return processed_text

    def _get_char_coords(self, char):
        for r in range(5):
            if char in self.key_table[r]:
                c = self.key_table[r].index(char)
                return r, c
        return -1, -1

    def encrypt(self, plaintext):
        prepared_text = self._prepare_text(plaintext)
        ciphertext = ""
        for i in range(0, len(prepared_text), 2):
            char1, char2 = prepared_text[i], prepared_text[i+1]
            r1, c1 = self._get_char_coords(char1)
            r2, c2 = self._get_char_coords(char2)

            if r1 == r2: # Same Row
                new_char1 = self.key_table[r1][(c1 + 1) % 5]
                new_char2 = self.key_table[r2][(c2 + 1) % 5]
            elif c1 == c2: # Same Column
                new_char1 = self.key_table[(r1 + 1) % 5][c1]
                new_char2 = self.key_table[(r2 + 1) % 5][c2]
            else: # Rectangle
                new_char1 = self.key_table[r1][c2]
                new_char2 = self.key_table[r2][c1]

            ciphertext += new_char1 + new_char2
        return ciphertext


# ------------------------------------
# 2. Vigenère Cipher Implementation
# ------------------------------------

class VigenereCipher:
    """Implements the Vigenère Cipher."""

    def __init__(self, key):
        self.key = self._clean_key(key)

    def _clean_text(self, text):
        return "".join(c for c in text.upper() if 'A' <= c <= 'Z')

    def _clean_key(self, key):
        cleaned_key = "".join(c for c in key.upper() if 'A' <= c <= 'Z')
        if not cleaned_key:
            raise ValueError("Vigenere key must contain at least one alphabetic character.")
        return cleaned_key

    def _generate_repeated_key(self, plaintext):
        cleaned_plaintext = self._clean_text(plaintext)
        key_length = len(self.key)
        full_repeats = len(cleaned_plaintext) // key_length
        remainder = len(cleaned_plaintext) % key_length
        repeated_key = (self.key * full_repeats) + self.key[:remainder]
        return repeated_key

    def encrypt(self, plaintext):
        cleaned_plaintext = self._clean_text(plaintext)
        repeated_key = self._generate_repeated_key(cleaned_plaintext)
        ciphertext = []

        for p_char, k_char in zip(cleaned_plaintext, repeated_key):
            p_val = ord(p_char) - ord('A')
            k_val = ord(k_char) - ord('A')
            c_val = (p_val + k_val) % 26
            c_char = chr(c_val + ord('A'))
            ciphertext.append(c_char)

        return "".join(ciphertext)


# ------------------------------------
# 3. Argon2 Password Hashing (Modern Technique)
# ------------------------------------

class Argon2PasswordManager:
    """Implements Argon2 for secure, one-way password hashing and verification."""

    def __init__(self):
        self.ph = PasswordHasher()

    def hash_password(self, password: str) -> str:
        return self.ph.hash(password)

    def verify_password(self, password: str, hashed_password: str) -> bool:
        """Correctly verifies the password, returning False on mismatch."""
        try:
            self.ph.verify(hashed_password, password)
            if self.ph.check_needs_rehash(hashed_password) and ARGON2_AVAILABLE:
                print("[INFO] Password hash needs update (rehash).")
            return True
        except VerifyMismatchError:
            return False
        except Exception as e:
            print(f"[ERROR] Verification failed due to internal error: {e}")
            return False


# ------------------------------------
# Main Application Runner
# ------------------------------------

def run_application():
    """Main function to run the password encryption demonstration."""

    print("===================================================================")
    print("       BINF711 - Milestone 2: Password Encryption Prototype")
    print("===================================================================")

    # Check for Argon2
    if not ARGON2_AVAILABLE:
        print("!! WARNING: Argon2 library not found. Using a MOCK implementation. !!")
        print("!! Run 'pip install argon2-cffi' for the actual required security. !!\n")

    # 1. Get common input
    password = input("Enter the password you wish to encrypt/hash: ")
    key = input("Enter a secret key (used for Playfair/Vigenere): ")

    if not key:
        print("\n[ERROR] Key cannot be empty. Exiting.")
        return

    # --- 1. Playfair Cipher Demonstration ---
    try:
        print("\n--- 1. Playfair Cipher ---")
        playfair = PlayfairCipher(key)
        playfair_ciphertext = playfair.encrypt(password)
        print(f"Prepared Text: {playfair._prepare_text(password)}")
        print(f"Ciphertext: {playfair_ciphertext}")
    except Exception as e:
        print(f"[ERROR] Playfair failed: {e}")

    # --- 2. Vigenère Cipher Demonstration ---
    try:
        print("\n--- 2. Vigenère Cipher ---")
        vigenere = VigenereCipher(key)
        vigenere_ciphertext = vigenere.encrypt(password)
        print(f"Ciphertext: {vigenere_ciphertext}")
    except Exception as e:
        print(f"[ERROR] Vigenère failed: {e}")

    # --- 3. Argon2 Password Hashing Demonstration (Modern) ---
    try:
        print("\n--- 3. Argon2 Hashing (Recommended Modern Method) ---")
        argon2_manager = Argon2PasswordManager()

        # Hashing
        argon2_hash = argon2_manager.hash_password(password)
        print(f"Generated Hash: {argon2_hash}")

        # Verification
        wrong_password = password + "z" # Creates a deliberately incorrect test password

        correct_check = argon2_manager.verify_password(password, argon2_hash)
        wrong_check = argon2_manager.verify_password(wrong_password, argon2_hash)

        print(f"Verification Check (Correct Password): {'SUCCESS' if correct_check else 'FAIL'}")
        print(f"Verification Check (Wrong Password): {'SUCCESS' if wrong_check else 'FAIL'}")

    except Exception as e:
        print(f"[ERROR] Argon2 failed. Check library installation: {e}")


if __name__ == "__main__":
    run_application()

       BINF711 - Milestone 2: Password Encryption Prototype
Enter the password you wish to encrypt/hash: youssef
Enter a secret key (used for Playfair/Vigenere): alo

--- 1. Playfair Cipher ---
Prepared Text: YOUSSEFX
Ciphertext: XBQTRFMO

--- 2. Vigenère Cipher ---
Ciphertext: YZISDSF

--- 3. Argon2 Hashing (Recommended Modern Method) ---
Generated Hash: $argon2id$v=19$m=65536,t=3,p=4$OjVLFH6iZr5ute+b94jHsA$Tm2dDfxsx8yvtYkS+KsAHUtGEZUfVgkGjQIaT07Hbqg
Verification Check (Correct Password): SUCCESS
Verification Check (Wrong Password): FAIL
